# MLP Bag of Words

En esta notebook, haremos un clasificador en base a una MLP + Bag of Words



In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import csv

df_dev = pd.read_table("../../../data/es/dev_es.tsv", index_col="id", quoting=csv.QUOTE_NONE)
df_train = pd.read_table("../../../data/es/train_es.tsv", index_col="id", quoting=csv.QUOTE_NONE)

text_train, y_train = df_train["text"], df_train["HS"]
text_dev, y_dev = df_dev["text"], df_dev["HS"]

print("Instancias de entrenamiento: {}".format(len(df_train)))
print("Instancias de desarrollo: {}".format(len(df_dev)))


Instancias de entrenamiento: 4500
Instancias de desarrollo: 500


In [3]:
from nltk.tokenize import TweetTokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem import SnowballStemmer
from gensim.utils import deaccent

stemmer = SnowballStemmer('spanish')

def tokenize(tweet, **kwargs):
    #tweet = deaccent(tweet)
    tokenizer = TweetTokenizer(**kwargs)
    tokens = tokenizer.tokenize(tweet)
    
    ret = []
    for token in tokens:
        if token[0] == "#":
            ret.append(stemmer.stem(token[1:]))
        elif token[0] == "@":
            ret.append("@user")
        else:
            ret.append(stemmer.stem(token))
    
    return ret

tokenize_args = {"preserve_case": False, "reduce_len": True, "strip_handles": False}

vect = TfidfVectorizer(tokenizer=lambda t: tokenize(t, **tokenize_args), 
                       max_df=0.90, min_df=0.001, ngram_range=(1, 2), binary=True)

vect.fit(df_train["text"])



X_train = vect.transform(df_train["text"])
X_dev = vect.transform(df_dev["text"])

X_train.shape, X_dev.shape

Using TensorFlow backend.


((4500, 4300), (500, 4300))

In [4]:
from keras.models import Model
from keras.layers import (
    Dense, GRU, Dropout, LSTM, Bidirectional, Embedding,
    CuDNNLSTM, Conv1D, MaxPooling1D, Input, Concatenate)
from keras.optimizers import Adam

optimizer_args = {
    "lr": 0.001,
    "decay": 0.01,
}


input_char = Input(shape=(X_train.shape[1],))
x = Dense(512, activation='relu')(input_char)
x = Dropout(0.75)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.55)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=[input_char], outputs=[output])
model.compile(loss='binary_crossentropy', 
              optimizer=Adam(**optimizer_args), 
              metrics=['accuracy'])

model.fit([X_train], y_train, validation_data=(X_dev, y_dev), epochs=10, batch_size=32)


Train on 4500 samples, validate on 500 samples
Epoch 1/10
4500/4500 [==============================] - 2s 494us/step - loss: 0.6494 - acc: 0.6047 - val_loss: 0.6011 - val_acc: 0.6520
Epoch 2/10
4500/4500 [==============================] - 1s 216us/step - loss: 0.4941 - acc: 0.7760 - val_loss: 0.4723 - val_acc: 0.7820
Epoch 3/10
4500/4500 [==============================] - 1s 217us/step - loss: 0.3738 - acc: 0.8467 - val_loss: 0.4419 - val_acc: 0.8000
Epoch 4/10
4500/4500 [==============================] - 1s 218us/step - loss: 0.3087 - acc: 0.8844 - val_loss: 0.4434 - val_acc: 0.7980
Epoch 5/10
4500/4500 [==============================] - 1s 220us/step - loss: 0.2716 - acc: 0.9009 - val_loss: 0.4430 - val_acc: 0.7980
Epoch 6/10
4500/4500 [==============================] - 1s 218us/step - loss: 0.2402 - acc: 0.9142 - val_loss: 0.4483 - val_acc: 0.7960
Epoch 7/10
4500/4500 [==============================] - 1s 218us/step - loss: 0.2122 - acc: 0.9276 - val_loss: 0.4553 - val_acc: 0.7960
E

In [4]:
from hate.utils import print_evaluation

print_evaluation(model, X_dev, y_dev)

500/500 [==============================] - 0s 95us/step
Loss        : 0.4917
Accuracy    : 0.8020
Precision   : 0.7834
Recall      : 0.7658
F1          : 0.7745


## Error Analysis

In [5]:
df_dev["proba"] = model.predict(X_dev)


true_positives = df_dev[(df_dev["HS"] == 1) & (df_dev["proba"] >= 0.5)].copy()
true_negatives = df_dev[(df_dev["HS"] == 0) & (df_dev["proba"] < 0.5)].copy()

false_positives = df_dev[(df_dev["HS"] == 0) & (df_dev["proba"] > 0.5)].copy()
false_positives.sort_values("proba", ascending=False, inplace=True)


false_negatives = df_dev[(df_dev["HS"] == 1) & (df_dev["proba"] < 0.5)].copy()
false_negatives.sort_values("proba", ascending=True, inplace=True)

conf_matrix = pd.DataFrame([
    {"real":"hs=1", "pred_true": len(true_positives), "pred_false": len(false_negatives)},
    {"real":"hs=0", "pred_true": len(false_positives), "pred_false": len(true_negatives)}
])



conf_matrix.set_index("real", inplace=True)

print("Falsos negativos: {}".format(len(false_negatives)))
print("Falsos positivos: {}".format(len(false_positives)))

conf_matrix[["pred_true", "pred_false"]]

Falsos negativos: 51
Falsos positivos: 46


,pred_true,pred_false
real,,
hs=1,171,51
hs=0,46,232


## Falsos Negativos


In [6]:
pd.set_option('max_colwidth', 300)

false_negatives[["text", "proba"]]


,text,proba
id,,
21072,"Voy andando por la calle y me han dicho cosas en árabe ya que me estas acosando dime lo en español para que te entienda, asique como yo no les entiendo me he cagado en su puta madre en árabe para que ellos a mi si.",0.001103
23716,"- El otro día me comí a Laura es muy fácil que guarra jajajaja - Laura también te comió a ti entonces tú eres igual de guarro e igual de fácil. - Puto feminazi putas modas eres feminista solo para ligar bien queda gilipollas - em, ok",0.002520
20711,@lopezdoriga eso es culpa de los gobiernos corruptos que tienen arreglos con esta escoria. Ojalá se muera la perra esa...,0.003656
21215,"Que tal otro aporte anónimo son de la puta de mi esposa, así los dejó en navidad enseñando le los calzones al vecino rt y siguenos @TANGASUCIA @Tangasrobadas https://t.co/70ioUjveqY",0.004956
24925,"La verdad que si, no veia la hora de llegar a los 5 mil amigos😂 ahora de a poco voy borrando esos indios indocumentados😂 https://t.co/Xw8q4GLM8v",0.008221
21152,@NatyBurgos15 @Horaciogenta @PaolaPa05 Cállate vos Black and bitch como la puta falsa abogada. Espero se pudra en la cárcel y devuelva todo. Dedícate a estudiar en serio. La droga es mala. Cristina puta,0.011846
22065,| lo único que dominas es mi polla en tu cara. https://t.co/qlYOd4zPwW,0.019471
22199,Todo aquél que se haya molestado en investigar un poco sabe que desde tiempos antiguos los mayores negreros y esclavistas del mundo fueron los árabes. https://t.co/DY5Uo8979v,0.033104
24055,@elisacarrio Callate gorda borracha hija de mil puta sos lo peor de la politica argentina ya lo dijo don Raul Alfonsin,0.041495


¿Cómo los tokenizamos?

# Conclusiones

- Modelo BoW con una MLP anda bastante bien

No estarían convenciendo las traducciones...

Diría que intentemos un ensemble con caracteres o n-gramas.